In [3]:
import os
import re
import numpy as np
import pandas as pd
import cv2
import subprocess
from functions import *
import PyPDF2
from datetime import datetime

ModuleNotFoundError: No module named 'numpy'

In [139]:
main_folder = r"C:\Users\Lenovo\OneDrive\Desktop\Folders\NaharOm\BSA\Main_Project\PdfData\East_Sun_Logistics\Report-20220405163727 april 2021\ExcelData"

In [140]:
excel_files = [f for f in os.listdir(main_folder) if f.endswith('.csv')]

excel_files_list = [os.path.join(main_folder, f) for f in excel_files]

# excel_files_list = excel_files_list[2:] + excel_files_list[:2]
excel_files_list

['C:\\Users\\Lenovo\\OneDrive\\Desktop\\Folders\\NaharOm\\BSA\\Main_Project\\PdfData\\East_Sun_Logistics\\Report-20220405163727 april 2021\\ExcelData\\page_1_output_0.csv',
 'C:\\Users\\Lenovo\\OneDrive\\Desktop\\Folders\\NaharOm\\BSA\\Main_Project\\PdfData\\East_Sun_Logistics\\Report-20220405163727 april 2021\\ExcelData\\page_2_output_0.csv',
 'C:\\Users\\Lenovo\\OneDrive\\Desktop\\Folders\\NaharOm\\BSA\\Main_Project\\PdfData\\East_Sun_Logistics\\Report-20220405163727 april 2021\\ExcelData\\page_3_output_0.csv',
 'C:\\Users\\Lenovo\\OneDrive\\Desktop\\Folders\\NaharOm\\BSA\\Main_Project\\PdfData\\East_Sun_Logistics\\Report-20220405163727 april 2021\\ExcelData\\page_4_output_0.csv']

In [236]:
excel_sheet_list = []
for file in excel_files_list:
    sheet = pd.read_csv(file,header=None)
    sheet.dropna(axis=0, how='all', inplace=True)
    sheet.reset_index(drop=True, inplace=True)
    if len(sheet) >= 5 or sheet.shape[1] >= 4:
        excel_sheet_list.append(sheet)

In [237]:
terminology = None
for i in range(len(excel_sheet_list)):
    for j,row in excel_sheet_list[i].iterrows():
        row_1_concatenated = ' '.join(row.dropna().astype(str))

        if "balance" in row_1_concatenated.lower() and (("description" in row_1_concatenated.lower() or "particular" in row_1_concatenated.lower() or "narration" in row_1_concatenated.lower() or "remarks" in row_1_concatenated.lower()) or ("date" in row_1_concatenated.lower())):
            if "date" not in row_1_concatenated.lower() and "date" in ' '.join(excel_sheet_list[i].iloc[j+1].dropna().astype(str)).lower():
                excel_sheet_list[i].iloc[j] = excel_sheet_list[i].iloc[j].astype(str) + ' ' + excel_sheet_list[i].iloc[j+1].astype(str)
                excel_sheet_list[i].drop(labels =j+1,inplace=True)
                excel_sheet_list[i].reset_index(drop=True, inplace=True)
            terminology = excel_sheet_list[i].iloc[j]
            break

    if terminology is not None:
        print(f"Terminology found in sheet {i+1}")
        break

Terminology found in sheet 1


In [238]:
terminology

0             SI. No.
1                 NaN
2                Date
3         Description
4                 NaN
5    Chq / Ref number
6              Amount
7              Dr/ Cr
8             Balance
9               Dr/Cr
Name: 0, dtype: object

In [239]:
for i in range(len(excel_sheet_list)):
    diff = excel_sheet_list[i].apply(lambda x : diff_first_last_non_nan(x),axis=1)
    excel_sheet_list[i] = excel_sheet_list[i][diff>2].copy()
    
    for j, row in excel_sheet_list[i].iterrows():
        if  row.dropna().apply(lambda x: is_pure_string(x)).all():
            excel_sheet_list[i].drop(labels = j,inplace=True)
    excel_sheet_list[i].reset_index(drop=True, inplace=True)

In [240]:
for i in range(len(excel_sheet_list)):
    for j, row in excel_sheet_list[i].iterrows():
        if not row.dropna().apply(lambda x: check_date(x)).any():
            excel_sheet_list[i].drop(labels = j,inplace=True)
    
    excel_sheet_list[i].dropna(axis=1, how='all', inplace=True)
    excel_sheet_list[i].reset_index(drop=True, inplace=True)
    excel_sheet_list[i].columns = range(excel_sheet_list[i].shape[1])     

In [241]:
non_nan_column_indexes = len(terminology_preprocessing(terminology).dropna())
dates_idx,dates_num,balance_idx,description_idx,convention = column_index_processed(terminology,excel_sheet_list)

dates_idx,dates_num,balance_idx,description_idx,convention

([1], 1, 6, 2, [True, [5, 4]])

In [242]:
for i in range(len(excel_sheet_list)):
    for j, row in excel_sheet_list[i].iterrows():
        # drop row if balance is not present
        if np.isnan(amount_parser(row[balance_idx])):
            excel_sheet_list[i].drop(labels = j,inplace=True)
    excel_sheet_list[i].reset_index(drop=True, inplace=True)

In [243]:
for i in range(len(excel_sheet_list)):
    if excel_sheet_list[i].shape[1] < non_nan_column_indexes:
        # add a nan column jst after description column
        if excel_sheet_list[i][dates_idx[0]].dropna().apply(lambda x: check_date(x)).all():
            pass
        elif dates_idx[0]>0 and excel_sheet_list[i][dates_idx[0]-1].dropna().apply(lambda x: check_date(x)).all():
            excel_sheet_list[i].insert(dates_idx[0]-1, 'nan', np.nan)
            excel_sheet_list[i].columns = range(excel_sheet_list[i].shape[1])
            continue

        if excel_sheet_list[i][description_idx].apply(lambda x: is_pure_string(x)).all():
            pass
        elif description_idx>0 and excel_sheet_list[i][description_idx-1].apply(lambda x: is_pure_string(x)).all(): # type: ignore
            excel_sheet_list[i].insert(description_idx-1, 'nan', np.nan) # type: ignore
            excel_sheet_list[i].columns = range(excel_sheet_list[i].shape[1])
            continue


        excel_sheet_list[i].insert(description_idx+1, 'nan', np.nan) # type: ignore
        excel_sheet_list[i].columns = range(excel_sheet_list[i].shape[1])
    
    elif excel_sheet_list[i].shape[1] > non_nan_column_indexes:
        # if there are zero values in balance, see if there is a column to right and assign it to balance
        # delete a column from right of description
        difference = excel_sheet_list[i].shape[1] - non_nan_column_indexes
        column_to_delete = [description_idx+1+i for i in range(difference)]
        excel_sheet_list[i].drop(columns=column_to_delete,inplace=True) # type: ignore
        excel_sheet_list[i].columns = range(excel_sheet_list[i].shape[1])

In [244]:
final_extracted_sheets = []
for i in range(len(excel_sheet_list)):
    extracted_sheet = excel_sheet_list[i].iloc[:,[dates_idx[0],description_idx,convention[1][0],convention[1][1],balance_idx]]

    final_extracted_sheets.append(extracted_sheet)

In [245]:
non_nan_column_indexes

8

In [246]:
excel_sheet_list[-1].shape[1]

8

In [247]:
excel_sheet_list[-2]

,0,1,2,3,4,5,6,7
0,44,13/04/2021,MB:SALARY FOR FEB AND,MB-999283056597,"20,000.00",DR,"358,412.69",CR
1,45,13/04/2021,UPI/VIGNESH,UPI-110313928129,262.00,DR,"378,412.69",CR
2,46,12/04/2021,Received from FRES XX0610,IMPS-110217154979,"200,000.00",CR,"378,674.69",CR
3,47,12/04/2021,FRESH N FROZEN,NEFTINW-0282710362,"100,000.00",CR,"178,674.69",CR
4,48,11/04/2021,MB:srf payment,000209753831,"120,000.00",DR,"78,674.69",CR
5,49,11/04/2021,MB:srf transporter fnf 126,000209735349,"50,000.00",DR,"198,674.69",CR
6,50,11/04/2021,UPI/Mohd llyas,UPI-110112183375,"50,000.00",CR,"248,674.69",CR
7,51,11/04/2021,Subh/110170078279/Paymen,UPI-110112999022,10.00,DR,"198,674.69",CR
8,52,11/04/2021,MONTH OF FEB,MB-999284030050,"15,000.00",DR,"198,684.69",CR
9,53,11/04/2021,Dated On 27-Mar-2021,TBMS-725756825,5.90,DR,"213,684.69",CR


In [248]:
last_sheets = extracted_sheet_processing(final_extracted_sheets,dates_idx,dates_num,balance_idx,description_idx,convention)

c:\Users\Lenovo\OneDrive\Desktop\Folders\NaharOm\BSA\Main_Project\functions.py:387: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  extracted_sheet['Date'] = extracted_sheet['Date'].apply(lambda x : pd.to_datetime(x,errors='coerce',dayfirst=True))
c:\Users\Lenovo\OneDrive\Desktop\Folders\NaharOm\BSA\Main_Project\functions.py:387: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  extracted_sheet['Date'] = extracted_sheet['Date'].apply(lambda x : pd.to_datetime(x,errors='coerce',dayfirst=True))


In [249]:
# concatenate all the sheets -
final_df = pd.concat(last_sheets,ignore_index=True)

In [251]:
# save the final df to a csv file

final_df.to_csv(os.path.join(main_folder,"final_extracted_data.csv"),index=False)